Changes were made in this repository by changing target classes as 9 --> 1 for binary classification.

Changes in autoattack.py, autopgd.py, fab_base.py

Changes in autoattack.py depending on the environment

In [ ]:
!git clone https://github.com/fra31/auto-attack

Cloning into 'auto-attack'...
remote: Enumerating objects: 672, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 672 (delta 162), reused 156 (delta 137), pack-reused 463 (from 1)
Receiving objects: 100% (672/672), 40.63 MiB | 39.22 MiB/s, done.
Resolving deltas: 100% (413/413), done.


In [2]:
import numpy as np
import pandas as pd
import pathlib
from pathlib import Path
import os
import zipfile
import torch
import torch.nn as nn
from torchvision.models import resnet50
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import shutil
import numpy as np
from PIL import Image

In [6]:
from autoattack import AutoAttack

In [3]:
root_dir = Path(os.getcwd())
output_dir = os.path.join(root_dir, "AutoAttack_Cifake")
os.makedirs(output_dir, exist_ok=True)
print(output_dir)
def directory_structure(base_dir):
    structure = [
        "train/REAL",
        "train/FAKE",
        "test/REAL",
        "test/FAKE"
    ]
    for subfolder in structure:
        os.makedirs(os.path.join(base_dir, subfolder), exist_ok=True)
directory_structure(output_dir)

/content/AutoAttack_Cifake


In [ ]:
#ADD the archive.zip file
zip_path =  os.path.join(root_dir, "archive.zip")
extract_path = os.path.join(root_dir, "CiFake")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"Files extracted to: {extract_path}")

In [ ]:
#File handling and sorting the first images in cifake
TRAINREAL = os.path.join(extract_path, "train/REAL")
TRAINFAKE = os.path.join(extract_path, "train/FAKE")
TESTFAKE = os.path.join(extract_path, "test/FAKE")
TESTREAL = os.path.join(extract_path, "test/REAL")
image_TRAINREAL = [f for f in os.listdir(TRAINREAL) if f.lower().endswith('.jpg')]
image_TRAINFAKE = [f for f in os.listdir(TRAINFAKE) if f.lower().endswith('.jpg')]
image_TESTFAKE = [f for f in os.listdir(TESTFAKE) if f.lower().endswith('.jpg')]
image_TESTREAL = [f for f in os.listdir(TESTREAL) if f.lower().endswith('.jpg')]
image_TRAINREAL.sort()
image_TRAINFAKE.sort()
image_TESTFAKE.sort()
image_TESTREAL.sort()
trreal = image_TRAINREAL[:3500]
trfake = image_TRAINFAKE[:3500]
ttreal = image_TESTREAL[:1000]
ttfake = image_TESTFAKE[:1000]
real_train = os.path.join(root_dir, "TRAIN1/REAL")
fake_train = os.path.join(root_dir,"TRAIN2/FAKE" )
fake_test = os.path.join(root_dir,"TEST2/FAKE" )
real_test = os.path.join(root_dir,"TEST1/REAL" )
print(real_train, fake_train)
os.makedirs(real_train, exist_ok=True)
os.makedirs(fake_train, exist_ok=True)
os.makedirs(fake_test, exist_ok=True)
os.makedirs(real_test, exist_ok=True)

In [5]:
os.chdir(os.path.join(root_dir, "auto-attack"))

In [8]:
#Add the weights fake_detector.pth
weights = os.path.join(root_dir, "fake_detector.pth")

In [ ]:
def load_Resnet50(model_path, num_classes=2, device='cuda'):
    """Loading the weights"""
    model = resnet50(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    state_dict = torch.load(model_path)
    if isinstance(state_dict, dict):
        new_state_dict = {}
        for k, v in state_dict.items():
            if k.startswith('module.'):
                new_state_dict[k[7:]] = v
            else:
                new_state_dict[k] = v
        model.load_state_dict(new_state_dict)
    else:
        model = state_dict

    return model.to(device)

classifier = load_Resnet50(weights)

In [ ]:
#input tensor
def INPUT_PATH(img_path):
    input_org_image = Image.open(img_path).resize((224, 224), Image.Resampling.LANCZOS)
    testX = np.array(input_org_image)/255.0
    model_input_org = testX[np.newaxis, ...]
    torch_testX = torch.from_numpy( np.transpose(model_input_org, (0, 3, 1, 2)) ).float().cuda()
    return torch_testX

In [ ]:
def label_Input(filepath):
    second_last_part = os.path.basename(os.path.dirname(filepath))
    if second_last_part.upper() == "REAL":
        return torch.tensor([1]).cuda()
    elif second_last_part.upper() == "FAKE":
        return torch.tensor([0]).cuda()
    else:
        raise ValueError(f"Unexpected directory name: {second_last_part}")

In [ ]:
#new folder
for image in trreal:
    shutil.copy(os.path.join(TRAINREAL, image), os.path.join(real_train, image))
for image in trfake:
    shutil.copy(os.path.join(TRAINFAKE, image), os.path.join(fake_train, image))
for image in ttfake:
    shutil.copy(os.path.join(TESTFAKE, image), os.path.join(fake_test, image))
for image in ttreal:
    shutil.copy(os.path.join(TESTREAL, image), os.path.join(real_test, image))

In [10]:
#perturbation functions
adversary = AutoAttack(classifier, norm='Linf', eps=8/255., version='standard')
classifier.cuda()
classifier.eval()

setting parameters for standard version


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
#all 4 folders train--> real, fake, test--> real, fake
def run_Attack_train_real(img_path, model, filename):
    adversary = AutoAttack(model, norm='Linf', eps=8/255., version='standard')
    x_adv = adversary.run_standard_evaluation(INPUT_PATH(img_path),torch.tensor([1]).cuda() , bs=1, return_labels=False)
    model_input_adv = np.moveaxis(x_adv.cpu().numpy(), 1, 3)
    img = Image.fromarray((model_input_adv[0] * 255).astype('uint8'))
    model_input_adv = img.resize((32, 32), Image.Resampling.LANCZOS)
    output_path = (os.path.join(output_dir, f'train/REAL/{filename}_AutoAttack.jpg'))
    plt.imsave(output_path, np.array(model_input_adv))
    print(output_path)

directory_for_test1 = real_train
for  files in os.listdir(directory_for_test1):
    file_name = os.path.splitext(files)[0]
    print(file_name)
    file_path = os.path.join(directory_for_test1, files)
    if os.path.isfile(file_path):
        run_Attack_train_real(file_path, classifier, file_name)

In [ ]:
def run_Attack_test_real(img_path, model, filename):
    adversary = AutoAttack(model, norm='Linf', eps=8/255., version='standard')
    x_adv = adversary.run_standard_evaluation(INPUT_PATH(img_path),torch.tensor([1]).cuda() , bs=1, return_labels=False)
    model_input_adv = np.moveaxis(x_adv.cpu().numpy(), 1, 3)
    img = Image.fromarray((model_input_adv[0] * 255).astype('uint8'))
    model_input_adv = img.resize((32, 32), Image.Resampling.LANCZOS)
    output_path = (os.path.join(output_dir, f'test/REAL/{filename}_AutoAttack.jpg'))
    plt.imsave(output_path, np.array(model_input_adv))
    print(output_path)

directory_for_test2 = real_test
for  files in os.listdir(directory_for_test2):
    file_name = os.path.splitext(files)[0]
    print(file_name)
    file_path = os.path.join(directory_for_test2, files)
    if os.path.isfile(file_path):
        run_Attack_test_real(file_path, classifier, file_name)

In [ ]:
def run_Attack_test_fake(img_path, model, filename):
    adversary = AutoAttack(model, norm='Linf', eps=8/255., version='standard')
    x_adv = adversary.run_standard_evaluation(INPUT_PATH(img_path),torch.tensor([0]).cuda() , bs=1, return_labels=False)
    model_input_adv = np.moveaxis(x_adv.cpu().numpy(), 1, 3)
    img = Image.fromarray((model_input_adv[0] * 255).astype('uint8'))
    model_input_adv = img.resize((32, 32), Image.Resampling.LANCZOS)
    output_path = (os.path.join(output_dir, f'test/FAKE/{filename}_AutoAttack.jpg'))
    plt.imsave(output_path, np.array(model_input_adv))
    print(output_path)

directory_for_test3 = fake_test
for  files in os.listdir(directory_for_test3):
    file_name = os.path.splitext(files)[0]
    print(file_name)
    file_path = os.path.join(directory_for_test3, files)
    if os.path.isfile(file_path):
        run_Attack_test_fake(file_path, classifier, file_name)

In [ ]:
def run_Attack_train_fake(img_path, model, filename):
    adversary = AutoAttack(model, norm='Linf', eps=8/255., version='standard')
    x_adv = adversary.run_standard_evaluation(INPUT_PATH(img_path),torch.tensor([0]).cuda() , bs=1, return_labels=False)
    model_input_adv = np.moveaxis(x_adv.cpu().numpy(), 1, 3)
    img = Image.fromarray((model_input_adv[0] * 255).astype('uint8'))
    model_input_adv = img.resize((32, 32), Image.Resampling.LANCZOS)
    output_path = (os.path.join(output_dir, f'train/FAKE/{filename}_AutoAttack.jpg'))
    plt.imsave(output_path, np.array(model_input_adv))
    print(output_path)

directory_for_test4 = fake_train
for  files in os.listdir(directory_for_test4):
    file_name = os.path.splitext(files)[0]
    print(file_name)
    file_path = os.path.join(directory_for_test4, files)
    if os.path.isfile(file_path):
        run_Attack_train_fake(file_path, classifier, file_name)

In [ ]:
zip_path =  os.path.join(root_dir, "AutoAttack_Cifake.zip")
shutil.make_archive(base_name=zip_path.replace('.zip', ''), format='zip', root_dir=output_dir)
print(f"Folder compressed into: {zip_path}")